In [ ]:
!pip install qiskit
!pip install qiskit_algorithms

In [ ]:
import numpy as np
from qiskit_algorithms import VarQRTE, TimeEvolutionProblem
from qiskit.circuit.library import ExcitationPreserving
from qiskit_algorithms.time_evolvers.variational import RealMcLachlanPrinciple
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp


var_principle = RealMcLachlanPrinciple()
estimator = StatevectorEstimator()
total_time = 5.0
evolution_timestep = 0.2

hamiltonian = SparsePauliOp.from_list([("ZI", 0.5), ("IZ", 0.5), ("XX", 0.2)])


def init_circ():
    qc = QuantumCircuit(2, 0)
    qc.x(0)  # initial state is |10>
    return qc


# Ansatz at t=0 must be equal to the Identity
params = [np.array([0.0 for i in range(5)])]
ansatz = ExcitationPreserving(num_qubits=2, entanglement="linear", reps=1)
ansatz = init_circ().compose(ansatz)

# Define and Run the Time Evolution Problem
evolution_problem = TimeEvolutionProblem(hamiltonian, total_time)
qrte = VarQRTE(ansatz, params[0][:], variational_principle=var_principle,
               estimator=estimator,
               num_timesteps=int(total_time / evolution_timestep),
)
params = qrte.evolve(evolution_problem).parameter_values

# Assemble the circuit which creates the evolved state
evolved_circ = ansatz.assign_parameters(params[-1])
print(evolved_circ)

In [ ]:
from qiskit.circuit.library import EfficientSU2
from scipy.optimize import minimize


# Estimate the energy using a quantum computer
def energy(params, ansatz, hamiltonian, estimator):
    current_energy = (
        estimator.run([(ansatz, hamiltonian, params)], precision=0.0001)
        .result()[0]
        .data.evs
    )
    print("Energy: " + str(current_energy))
    return current_energy


estimator = StatevectorEstimator()

hamiltonian = SparsePauliOp.from_list([("ZI", 0.5), ("IZ", 0.5), ("XX", 0.2)])

# Define the ansatz which will be used to prepare the ground state
ansatz = EfficientSU2(hamiltonian.num_qubits)

initial_params = np.random.random(ansatz.num_parameters)

# Now run the minimization algorithm to minimize the energy
res = minimize(energy, initial_params, args=(ansatz, hamiltonian, estimator),
               method="cobyla"
)

# Lastly, output the energy estimate obtained
final_params = res.x
print(f"Final Energy Estimate: {(energy(final_params, ansatz, hamiltonian, estimator))}")